# Generate TypeScript Types from cards_i18n.json

This script will analyze the structure of cards_i18n.json and generate comprehensive TypeScript type definitions with proper enum-like types for fields like cardTypeCode, colorCode, etc.

In [2]:
import json
import os
from typing import Dict, Any, Set, List, Optional
from collections import defaultdict

# Known enum-like fields that should be represented as union types
ENUM_FIELDS = {
    'cardTypeCode': set(),
    'colorCode': set(),
    'bloomLevelCode': set(),
    'rarityCode': set(),
    'timingCode': set()
}

# Known mappings for enum values
ENUM_MAPPINGS = {
    'cardTypeCode': {
        'character': 'character',
        'buzzCharacter': 'buzzCharacter',
        'oshiCharacter': 'oshiCharacter',
        'supportEvent': 'supportEvent',
        'supportEventLimited': 'supportEventLimited',
        'supportItem': 'supportItem',
        'supportFan': 'supportFan',
        'supportTool': 'supportTool',
        'supportLocation': 'supportLocation',
        'supportCheer': 'supportCheer'
    },
    'colorCode': {
        'white': 'white',
        'red': 'red',
        'blue': 'blue',
        'green': 'green',
        'yellow': 'yellow',
        'purple': 'purple'
    },
    'bloomLevelCode': {
        'debut': 'debut',
        'first': 'first',
        'second': 'second'
    },
    'timingCode': {
        'once_per_turn': 'once_per_turn',
        'once_per_game': 'once_per_game'
    }
}

def get_type_of_value(value: Any) -> str:
    """Determine the TypeScript type of a given value"""
    if value is None:
        return 'null'
    elif isinstance(value, bool):
        return 'boolean'
    elif isinstance(value, int):
        return 'number'
    elif isinstance(value, float):
        return 'number'
    elif isinstance(value, str):
        return 'string'
    elif isinstance(value, list):
        if not value:
            return 'any[]'  # Empty array
        types = set(get_type_of_value(item) for item in value)
        if len(types) == 1:
            return f'{next(iter(types))}[]'
        else:
            return f'({" | ".join(sorted(types))})[]'
    elif isinstance(value, dict):
        return 'object'
    else:
        return 'any'

def collect_enum_values(data: List[Dict[str, Any]]) -> Dict[str, Set[str]]:
    """Collect all possible values for enum-like fields"""
    enum_values = defaultdict(set)
    
    for card in data:
        for enum_field in ENUM_FIELDS.keys():
            # Handle nested fields like oshiSkill.timingCode
            if '.' in enum_field:
                parent, child = enum_field.split('.')
                if parent in card and isinstance(card[parent], dict) and child in card[parent]:
                    value = card[parent][child]
                    if isinstance(value, str):
                        enum_values[enum_field].add(value)
            elif enum_field in card and card[enum_field] is not None:
                value = card[enum_field]
                if isinstance(value, str):
                    enum_values[enum_field].add(value)
        
        # Check nested timingCode in oshiSkill and spOshiSkill
        for skill_field in ['oshiSkill', 'spOshiSkill']:
            if skill_field in card and isinstance(card[skill_field], dict) and 'timingCode' in card[skill_field]:
                value = card[skill_field]['timingCode']
                if isinstance(value, str):
                    enum_values['timingCode'].add(value)
    
    return enum_values

def analyze_object_structure(obj: Dict[str, Any], prefix: str = '', collected_types: Dict[str, Dict[str, Set[str]]] = None, depth: int = 0) -> Dict[str, Dict[str, Set[str]]]:
    """Recursively analyze the structure of an object to determine field types"""
    if collected_types is None:
        collected_types = {}
    
    if isinstance(obj, dict):
        # Create a type name for this level
        type_name = prefix if prefix else 'Card'
        
        # Initialize the type definition if not already present
        if type_name not in collected_types:
            collected_types[type_name] = {}
        
        # Analyze each field in the object
        for key, value in obj.items():
            field_type = get_type_of_value(value)
            
            # For objects, we need to recurse deeper
            if field_type == 'object':
                # Generate a new type name for this nested object
                new_prefix = f"{prefix}{key[0].upper()}{key[1:]}" if prefix else f"{key[0].upper()}{key[1:]}"
                
                # Recursively analyze the nested object
                analyze_object_structure(value, new_prefix, collected_types, depth + 1)
                
                # Reference the new type in the parent
                if key not in collected_types[type_name]:
                    collected_types[type_name][key] = set()
                collected_types[type_name][key].add(new_prefix)
            
            # For arrays of objects, we need to analyze the array items
            elif field_type.endswith('[]') and field_type != 'any[]' and isinstance(value, list) and value and isinstance(value[0], dict):
                # Generate a type name for the array items
                item_prefix = f"{prefix}{key[0].upper()}{key[1:]}Item" if prefix else f"{key[0].upper()}{key[1:]}Item"
                
                # Analyze a sample item from the array
                analyze_object_structure(value[0], item_prefix, collected_types, depth + 1)
                
                # Reference the item type in the parent
                if key not in collected_types[type_name]:
                    collected_types[type_name][key] = set()
                collected_types[type_name][key].add(f"{item_prefix}[]")
            
            # For simple types, just record the type
            else:
                if key not in collected_types[type_name]:
                    collected_types[type_name][key] = set()
                collected_types[type_name][key].add(field_type)
    
    return collected_types

def generate_typescript_types(types_dict: Dict[str, Dict[str, Set[str]]], enum_values: Dict[str, Set[str]]) -> str:
    """Generate TypeScript type definitions from the analyzed structure"""
    output = []
    
    # Generate enum-like types first
    for enum_field, values in enum_values.items():
        if values:  # Only generate if we have values
            # If we have predefined mappings, use those instead
            if enum_field in ENUM_MAPPINGS:
                values = set(ENUM_MAPPINGS[enum_field].keys())
            
            type_name = enum_field[0].upper() + enum_field[1:] + 'Type'
            values_str = ' | '.join([f"'{value}'" for value in sorted(values)])
            output.append(f"export type {type_name} = {values_str};\n")
    
    # Sort the types by name to ensure consistent output
    for type_name in sorted(types_dict.keys()):
        fields = types_dict[type_name]
        
        output.append(f"export type {type_name} = {{")
        
        # Sort fields by name for consistent output
        for field_name in sorted(fields.keys()):
            field_types = fields[field_name]
            
            # Replace with enum-like type if applicable
            if field_name in enum_values and enum_values[field_name]:
                type_str = field_name[0].upper() + field_name[1:] + 'Type'
            # Special case for nested timing codes
            elif field_name == 'timingCode':
                type_str = 'TimingCodeType'
            else:
                # Join multiple possible types with a union type operator
                type_str = " | ".join(sorted(field_types))
            
            # Add question mark for fields that might be undefined (optional)
            # A field is optional if it's not always present in all objects of this type
            is_optional = 'null' in field_types or len(field_types) > 1 and 'undefined' in field_types
            output.append(f"  {field_name}{'' if is_optional else '?'}: {type_str};")
        
        output.append("}\n")
    
    return "\n".join(output)

def process_json_file(input_file='../data-grab-scripts/cards_i18n.json', output_file='../types/card.ts'):
    """Process the cards_i18n.json file and generate TypeScript types"""
    try:
        # Load the JSON file
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Collect enum values from all cards
        enum_values = collect_enum_values(data)
        print("Collected enum values:")
        for field, values in enum_values.items():
            print(f"{field}: {sorted(values)}")
        
        # Analyze the structure using a sample card
        if data and isinstance(data, list) and len(data) > 0:
            # Merge multiple cards to ensure we capture all possible fields
            merged_card = {}
            for card in data:
                for key, value in card.items():
                    if key not in merged_card or value is not None:
                        merged_card[key] = value
            
            # Analyze the merged card
            types_dict = analyze_object_structure(merged_card)
            
            # Generate TypeScript types
            typescript_types = generate_typescript_types(types_dict, enum_values)
            
            # Add a type for the array of cards
            typescript_types += "\nexport type CardCollection = Card[];\n"
            
            print("\nGenerated TypeScript types:")
            print(typescript_types)
            
            # Write the types to a file
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(typescript_types)
            
            print(f"\nTypeScript types have been written to {output_file}")
        else:
            print("The JSON file is empty or not an array.")
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
    except json.JSONDecodeError:
        print(f"Error: File '{input_file}' contains invalid JSON.")
    except Exception as e:
        print(f"Error: {str(e)}")

# Run the processing function
process_json_file()

Collected enum values:
cardTypeCode: ['buzzCharacter', 'character', 'oshiCharacter', 'supportEvent', 'supportEventLimited', 'supportFan', 'supportItem', 'supportTool', 'unknown']
colorCode: ['blue', 'green', 'purple', 'red', 'unknown', 'white', 'yellow']
rarityCode: ['C', 'OC', 'OSR', 'OUR', 'P', 'R', 'RR', 'S', 'SEC', 'SR', 'SY', 'U', 'UR']
timingCode: ['once_per_game', 'once_per_turn']
bloomLevelCode: ['debut', 'first', 'second', 'unknown']

Generated TypeScript types:
export type CardTypeCodeType = 'buzzCharacter' | 'character' | 'oshiCharacter' | 'supportCheer' | 'supportEvent' | 'supportEventLimited' | 'supportFan' | 'supportItem' | 'supportLocation' | 'supportTool';

export type ColorCodeType = 'blue' | 'green' | 'purple' | 'red' | 'white' | 'yellow';

export type RarityCodeType = 'C' | 'OC' | 'OSR' | 'OUR' | 'P' | 'R' | 'RR' | 'S' | 'SEC' | 'SR' | 'SY' | 'U' | 'UR';

export type TimingCodeType = 'once_per_game' | 'once_per_turn';

export type BloomLevelCodeType = 'debut' | 'firs